# Note

#### 1. use kernel_initializer = 'ones' instead of 'zeros' -> the gradients do not flow through the ReLu layer

In [1]:
import numpy as np

In [2]:
# create dummy files
np.random.seed(42)
EMBEDDING_SIZE = 15
X = np.random.randint(2, size=(1000,100))
X_val = np.random.randint(2, size=(1000,100))
y = np.random.randint(2, size=1000)
y_val = np.random.randint(2, size=1000)

In [3]:
X.shape

(1000, 100)

In [4]:
y.shape

(1000,)

In [5]:
from keras.layers import Input, Dense, TimeDistributed, Embedding
from keras.layers import Concatenate, Reshape, Lambda, Multiply, multiply, concatenate, ThresholdedReLU
from keras.models import Model
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [6]:
y_tanh = []
y_tanh_val = []

for i in y:
    if i==0:
        y_tanh.append(-1)
    else:
        y_tanh.append(1)


for i in y_val:
    if i==0:
        y_tanh_val.append(-1)
    else:
        y_tanh_val.append(1)
        
y_tanh = np.asarray(y_tanh)  
y_tanh_val = np.asarray(y_tanh_val)

In [7]:
# make first model

def build_base_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    tanh_output = Dense(1, activation='tanh', name='tanh_output')(input_layer)
    
    model = Model(inputs=input_layer, outputs=tanh_output)
    model.summary()
    return model

base_model = build_base_model(X.shape[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 101       
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Combined model

combined_input_layer = Input(shape=(X.shape[1],))

ht_1 = Input(shape=(1,))
ht_2 = Input(shape=(1,))

label_layer = base_model(combined_input_layer)

dense_1 = Dense(1, activation='relu')(Multiply()([ht_1, label_layer]))
dense_2 = Dense(1, activation='relu')(Multiply()([ht_2, label_layer]))

concat = Concatenate()([dense_1, dense_2])

output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_1, ht_2], outputs=output_layer)
combined_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            101         input_2[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
multiply_1

In [9]:
base_model.compile(loss='mse',
            optimizer='adam',
            metrics=['acc'])

In [10]:
combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [11]:
# make second model
# first, make the vector to see if it is appears 

# we have two terms (human-terms) for now. c1 == pos word, c2 == neg word
# 1st column gives the feature index where that particular human_terms located
# 2nd column gives whether it is positive/negative word
term = [[1,1],[2,-1]]
term = np.asarray(term)

#assume X is a document vector [1,n] where n is the number of terms

def np_present_by_term(X, term):
    np_vector = np.zeros([term.shape[0],1])
    
    for i,(feature_index,np_term) in enumerate(term):
        if X[feature_index] == 1:
            np_vector[i] = np_term
    
    return np_vector

# Assuming 'term' is a [1X2] vector
def np_present_by_doc(X, term):
    np_vector = np.zeros([X.shape[0],1])
    
    for i, x in enumerate(X):
        if x[term[0]] == 1:
            np_vector[i] = term[1]
    
    return np_vector

In [12]:
ht_1_input = np_present_by_doc(X, term[0])
ht_2_input = np_present_by_doc(X, term[1])

ht_1_input_val = np_present_by_doc(X_val, term[0])
ht_2_input_val = np_present_by_doc(X_val, term[1])

In [14]:
base_model.fit(X, y_tanh, validation_data=(X_val, y_tanh_val), epochs=500, verbose=1)

Train on 1000 samples, validate on 1000 samples
Epoch 1/500
1000/1000 [==============================] - 0s 30us/step - loss: 1.0916 - acc: 0.2130 - val_loss: 1.2172 - val_acc: 0.1740
Epoch 2/500
1000/1000 [==============================] - 0s 31us/step - loss: 1.0863 - acc: 0.2090 - val_loss: 1.2118 - val_acc: 0.1700
Epoch 3/500
1000/1000 [==============================] - 0s 31us/step - loss: 1.0680 - acc: 0.2130 - val_loss: 1.1992 - val_acc: 0.1660
Epoch 4/500
1000/1000 [==============================] - 0s 30us/step - loss: 1.0545 - acc: 0.2000 - val_loss: 1.1890 - val_acc: 0.1640
Epoch 5/500
1000/1000 [==============================] - 0s 28us/step - loss: 1.0439 - acc: 0.1960 - val_loss: 1.1838 - val_acc: 0.1560
Epoch 6/500
1000/1000 [==============================] - 0s 45us/step - loss: 1.0331 - acc: 0.1920 - val_loss: 1.1848 - val_acc: 0.1510
Epoch 7/500
1000/1000 [==============================] - 0s 34us/step - loss: 1.0240 - acc: 0.1960 - val_loss: 1.1801 - val_acc: 0.1510


1000/1000 [==============================] - 0s 30us/step - loss: 0.8939 - acc: 0.1290 - val_loss: 1.1270 - val_acc: 0.0840
Epoch 61/500
1000/1000 [==============================] - 0s 31us/step - loss: 0.8904 - acc: 0.1230 - val_loss: 1.1557 - val_acc: 0.0920
Epoch 62/500
1000/1000 [==============================] - 0s 31us/step - loss: 0.8911 - acc: 0.1360 - val_loss: 1.1267 - val_acc: 0.0840
Epoch 63/500
1000/1000 [==============================] - 0s 47us/step - loss: 0.8953 - acc: 0.1180 - val_loss: 1.1345 - val_acc: 0.0870
Epoch 64/500
1000/1000 [==============================] - 0s 29us/step - loss: 0.8895 - acc: 0.1210 - val_loss: 1.1388 - val_acc: 0.0860
Epoch 65/500
1000/1000 [==============================] - 0s 43us/step - loss: 0.8915 - acc: 0.1200 - val_loss: 1.1359 - val_acc: 0.0860
Epoch 66/500
1000/1000 [==============================] - 0s 38us/step - loss: 0.8937 - acc: 0.1210 - val_loss: 1.1270 - val_acc: 0.0860
Epoch 67/500
1000/1000 [==============================

1000/1000 [==============================] - 0s 37us/step - loss: 0.8895 - acc: 0.1470 - val_loss: 1.1407 - val_acc: 0.0890
Epoch 120/500
1000/1000 [==============================] - 0s 31us/step - loss: 0.8891 - acc: 0.1270 - val_loss: 1.1690 - val_acc: 0.1010
Epoch 121/500
1000/1000 [==============================] - 0s 32us/step - loss: 0.8923 - acc: 0.1490 - val_loss: 1.1357 - val_acc: 0.0850
Epoch 122/500
1000/1000 [==============================] - 0s 35us/step - loss: 0.8942 - acc: 0.1260 - val_loss: 1.1391 - val_acc: 0.0870
Epoch 123/500
1000/1000 [==============================] - 0s 46us/step - loss: 0.8897 - acc: 0.1310 - val_loss: 1.1445 - val_acc: 0.0920
Epoch 124/500
1000/1000 [==============================] - 0s 41us/step - loss: 0.8891 - acc: 0.1360 - val_loss: 1.1443 - val_acc: 0.0920
Epoch 125/500
1000/1000 [==============================] - 0s 40us/step - loss: 0.8909 - acc: 0.1250 - val_loss: 1.1423 - val_acc: 0.0910
Epoch 126/500
1000/1000 [=======================

1000/1000 [==============================] - 0s 31us/step - loss: 0.8918 - acc: 0.1350 - val_loss: 1.1693 - val_acc: 0.1000
Epoch 179/500
1000/1000 [==============================] - 0s 39us/step - loss: 0.8911 - acc: 0.1440 - val_loss: 1.1444 - val_acc: 0.0910
Epoch 180/500
1000/1000 [==============================] - 0s 33us/step - loss: 0.8891 - acc: 0.1370 - val_loss: 1.1575 - val_acc: 0.0970
Epoch 181/500
1000/1000 [==============================] - 0s 49us/step - loss: 0.8897 - acc: 0.1390 - val_loss: 1.1442 - val_acc: 0.0890
Epoch 182/500
1000/1000 [==============================] - 0s 43us/step - loss: 0.8914 - acc: 0.1320 - val_loss: 1.1499 - val_acc: 0.0960
Epoch 183/500
1000/1000 [==============================] - 0s 37us/step - loss: 0.8886 - acc: 0.1320 - val_loss: 1.1550 - val_acc: 0.0930
Epoch 184/500
1000/1000 [==============================] - 0s 30us/step - loss: 0.8893 - acc: 0.1340 - val_loss: 1.1474 - val_acc: 0.0950
Epoch 185/500
1000/1000 [=======================

1000/1000 [==============================] - 0s 34us/step - loss: 0.8915 - acc: 0.1290 - val_loss: 1.1760 - val_acc: 0.1030
Epoch 238/500
1000/1000 [==============================] - 0s 37us/step - loss: 0.8938 - acc: 0.1400 - val_loss: 1.1466 - val_acc: 0.0920
Epoch 239/500
1000/1000 [==============================] - 0s 31us/step - loss: 0.8881 - acc: 0.1390 - val_loss: 1.1504 - val_acc: 0.0940
Epoch 240/500
1000/1000 [==============================] - 0s 32us/step - loss: 0.8896 - acc: 0.1320 - val_loss: 1.1608 - val_acc: 0.1020
Epoch 241/500
1000/1000 [==============================] - 0s 42us/step - loss: 0.8925 - acc: 0.1450 - val_loss: 1.1472 - val_acc: 0.0950
Epoch 242/500
1000/1000 [==============================] - 0s 38us/step - loss: 0.8899 - acc: 0.1280 - val_loss: 1.1676 - val_acc: 0.1010
Epoch 243/500
1000/1000 [==============================] - 0s 37us/step - loss: 0.8915 - acc: 0.1540 - val_loss: 1.1496 - val_acc: 0.0940
Epoch 244/500
1000/1000 [=======================

1000/1000 [==============================] - 0s 39us/step - loss: 0.8896 - acc: 0.1280 - val_loss: 1.1522 - val_acc: 0.0980
Epoch 297/500
1000/1000 [==============================] - 0s 33us/step - loss: 0.8972 - acc: 0.1500 - val_loss: 1.1442 - val_acc: 0.0890
Epoch 298/500
1000/1000 [==============================] - 0s 33us/step - loss: 0.8909 - acc: 0.1360 - val_loss: 1.1490 - val_acc: 0.0910
Epoch 299/500
1000/1000 [==============================] - 0s 30us/step - loss: 0.8906 - acc: 0.1340 - val_loss: 1.1540 - val_acc: 0.0990
Epoch 300/500
1000/1000 [==============================] - 0s 39us/step - loss: 0.8938 - acc: 0.1370 - val_loss: 1.1817 - val_acc: 0.1060
Epoch 301/500
1000/1000 [==============================] - 0s 38us/step - loss: 0.8926 - acc: 0.1470 - val_loss: 1.1486 - val_acc: 0.0940
Epoch 302/500
1000/1000 [==============================] - 0s 38us/step - loss: 0.8882 - acc: 0.1310 - val_loss: 1.1602 - val_acc: 0.0990
Epoch 303/500
1000/1000 [=======================

1000/1000 [==============================] - 0s 30us/step - loss: 0.8903 - acc: 0.1350 - val_loss: 1.1453 - val_acc: 0.0910
Epoch 356/500
1000/1000 [==============================] - 0s 36us/step - loss: 0.8893 - acc: 0.1310 - val_loss: 1.1662 - val_acc: 0.1010
Epoch 357/500
1000/1000 [==============================] - 0s 42us/step - loss: 0.8914 - acc: 0.1360 - val_loss: 1.1522 - val_acc: 0.0960
Epoch 358/500
1000/1000 [==============================] - 0s 38us/step - loss: 0.8899 - acc: 0.1390 - val_loss: 1.1498 - val_acc: 0.0950
Epoch 359/500
1000/1000 [==============================] - 0s 37us/step - loss: 0.8939 - acc: 0.1430 - val_loss: 1.1463 - val_acc: 0.0910
Epoch 360/500
1000/1000 [==============================] - 0s 39us/step - loss: 0.8889 - acc: 0.1400 - val_loss: 1.1503 - val_acc: 0.0960
Epoch 361/500
1000/1000 [==============================] - 0s 37us/step - loss: 0.8888 - acc: 0.1370 - val_loss: 1.1546 - val_acc: 0.0950
Epoch 362/500
1000/1000 [=======================

1000/1000 [==============================] - 0s 32us/step - loss: 0.8902 - acc: 0.1340 - val_loss: 1.1528 - val_acc: 0.0930
Epoch 415/500
1000/1000 [==============================] - ETA: 0s - loss: 0.9583 - acc: 0.187 - 0s 46us/step - loss: 0.8902 - acc: 0.1390 - val_loss: 1.1570 - val_acc: 0.0980
Epoch 416/500
1000/1000 [==============================] - 0s 35us/step - loss: 0.8943 - acc: 0.1490 - val_loss: 1.1428 - val_acc: 0.0900
Epoch 417/500
1000/1000 [==============================] - 0s 32us/step - loss: 0.8907 - acc: 0.1450 - val_loss: 1.1586 - val_acc: 0.1000
Epoch 418/500
1000/1000 [==============================] - 0s 37us/step - loss: 0.8902 - acc: 0.1390 - val_loss: 1.1491 - val_acc: 0.0930
Epoch 419/500
1000/1000 [==============================] - 0s 37us/step - loss: 0.8884 - acc: 0.1340 - val_loss: 1.1682 - val_acc: 0.1000
Epoch 420/500
1000/1000 [==============================] - 0s 37us/step - loss: 0.8936 - acc: 0.1380 - val_loss: 1.1574 - val_acc: 0.0980
Epoch 421/

1000/1000 [==============================] - 0s 43us/step - loss: 0.8892 - acc: 0.1300 - val_loss: 1.1530 - val_acc: 0.0980
Epoch 474/500
1000/1000 [==============================] - 0s 42us/step - loss: 0.8888 - acc: 0.1380 - val_loss: 1.1452 - val_acc: 0.0890
Epoch 475/500
1000/1000 [==============================] - 0s 39us/step - loss: 0.8945 - acc: 0.1330 - val_loss: 1.1444 - val_acc: 0.0890
Epoch 476/500
1000/1000 [==============================] - 0s 41us/step - loss: 0.8907 - acc: 0.1390 - val_loss: 1.1520 - val_acc: 0.0960
Epoch 477/500
1000/1000 [==============================] - 0s 39us/step - loss: 0.8929 - acc: 0.1420 - val_loss: 1.1559 - val_acc: 0.0980
Epoch 478/500
1000/1000 [==============================] - 0s 35us/step - loss: 0.8888 - acc: 0.1380 - val_loss: 1.1499 - val_acc: 0.0970
Epoch 479/500
1000/1000 [==============================] - 0s 31us/step - loss: 0.8882 - acc: 0.1340 - val_loss: 1.1550 - val_acc: 0.0970
Epoch 480/500
1000/1000 [=======================

In [14]:
combined_model.fit([X, ht_1_input, ht_2_input], y, 
                   validation_data=([X_val, ht_1_input_val, ht_2_input_val], y_val), 
                   epochs=10, verbose=1)

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 392us/step - loss: 0.2499 - acc: 0.5040 - val_loss: 0.2500 - val_acc: 0.4820
Epoch 2/10
1000/1000 [==============================] - 0s 99us/step - loss: 0.2496 - acc: 0.5440 - val_loss: 0.2502 - val_acc: 0.4800
Epoch 3/10
1000/1000 [==============================] - 0s 93us/step - loss: 0.2493 - acc: 0.5380 - val_loss: 0.2505 - val_acc: 0.4690
Epoch 4/10
1000/1000 [==============================] - 0s 94us/step - loss: 0.2491 - acc: 0.5410 - val_loss: 0.2507 - val_acc: 0.4730
Epoch 5/10
1000/1000 [==============================] - 0s 96us/step - loss: 0.2489 - acc: 0.5330 - val_loss: 0.2510 - val_acc: 0.4700
Epoch 6/10
1000/1000 [==============================] - 0s 94us/step - loss: 0.2485 - acc: 0.5410 - val_loss: 0.2511 - val_acc: 0.4730
Epoch 7/10
1000/1000 [==============================] - 0s 92us/step - loss: 0.2482 - acc: 0.5440 - val_loss: 0.2513 - val_acc: 0.4710
Epoch 

### Simple Domain Test

In [15]:
# Generate 16 documents
# n1 = negative word with importance 1
# p2 = positive word with importance 2
# p3 = positive word with importance 3
# n4 = negative word with importance 4
# movie = neutral word with importance 0

word_list = ['movie', 'bad', 'good', 'amazing', 'awful']
word_weight = np.asarray([0, -1, 2, 3, -4], dtype='int32')
word_weight.shape

(5,)

In [16]:
from itertools import combinations

num = 16
data = np.zeros([num, len(word_list)])
index=0
label = []

for i in range(5):
    # get the combination
    perm = combinations([1,2,3,4], i)
    
    for j in list(perm):
        # always put neutral word
        data[index, 0] = 1
        
        # if its only 1 terms
        if i==1:
            data[index,j] = 1
        else:
            for k in j:
                data[index, k] = 1
        
        # calculate weights
        
        weights = data[index,:] * word_weight
        label.append(np.sign(np.sum(weights)))
        
        index+=1
        
label = np.asarray(label)

In [17]:
np_label = (label == 1).astype('int32')
np_label

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0])

In [18]:
label

array([ 0., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1.,
       -1.,  1.,  0.])

In [19]:
data.shape[0]

16

In [20]:
# build the combined model
# Combined model
###########################################
human_terms_len = 4
thresholded_relu = ThresholdedReLU(theta=0.9)
###########################################

base_model = build_base_model(data.shape[1])

combined_input_layer = Input(shape=(data.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

# get the document prediction
label_layer = base_model(combined_input_layer)

# multiply and pass it into relu
# initialize relu layer

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
    dense_layer.append(Dense(1, 
                             activation='relu', 
                             use_bias=False, 
                             kernel_initializer='ones')(Multiply()([split[i], label_layer])))

# concat all the result   
concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid', use_bias=False)(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 5)                 0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 5)            0                                            
_________________________________________________

In [21]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [22]:
def np_present_by_doc(X, index, weight):
    np_vector = np.zeros([X.shape[0],1])
    
    for i, x in enumerate(X):
        if x[index] == 1:
            np_vector[i] = np.sign(weight)
#             np_vector[i] = 1
    
    return np_vector

In [23]:
# I need to generate the ht's

ht_1_input = np_present_by_doc(data, 1, word_weight[1])
ht_2_input = np_present_by_doc(data, 2, word_weight[2])
ht_3_input = np_present_by_doc(data, 3, word_weight[3])
ht_4_input = np_present_by_doc(data, 4, word_weight[4])

In [24]:
ht_1_input.shape

(16, 1)

In [25]:
ht_input = np.hstack([ht_1_input, ht_2_input, ht_3_input, ht_4_input])

In [26]:
ht_input

array([[ 0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0., -1.],
       [-1.,  1.,  0.,  0.],
       [-1.,  0.,  1.,  0.],
       [-1.,  0.,  0., -1.],
       [ 0.,  1.,  1.,  0.],
       [ 0.,  1.,  0., -1.],
       [ 0.,  0.,  1., -1.],
       [-1.,  1.,  1.,  0.],
       [-1.,  1.,  0., -1.],
       [-1.,  0.,  1., -1.],
       [ 0.,  1.,  1., -1.],
       [-1.,  1.,  1., -1.]])

In [29]:
for i in range(len(ht_1_input)):
    print(data[i], ht_3_input[i])

[1. 0. 0. 0. 0.] [0.]
[1. 1. 0. 0. 0.] [0.]
[1. 0. 1. 0. 0.] [0.]
[1. 0. 0. 1. 0.] [1.]
[1. 0. 0. 0. 1.] [0.]
[1. 1. 1. 0. 0.] [0.]
[1. 1. 0. 1. 0.] [1.]
[1. 1. 0. 0. 1.] [0.]
[1. 0. 1. 1. 0.] [1.]
[1. 0. 1. 0. 1.] [0.]
[1. 0. 0. 1. 1.] [1.]
[1. 1. 1. 1. 0.] [1.]
[1. 1. 1. 0. 1.] [0.]
[1. 1. 0. 1. 1.] [1.]
[1. 0. 1. 1. 1.] [1.]
[1. 1. 1. 1. 1.] [1.]


In [28]:
# # adjust y to -> [-1, 1]
# label_tanh = []
# for i in label:
#     if i==0:
#         label_tanh.append(-1)
#     else:
#         label_tanh.append(1)
# label_tanh = np.asarray(label_tanh)

In [30]:
base_model.fit(data[1:15], label[1:15], epochs=1000, batch_size=1, verbose=2)

Epoch 1/1000
 - 0s - loss: 1.4815 - acc: 0.2143
Epoch 2/1000
 - 0s - loss: 1.4737 - acc: 0.2143
Epoch 3/1000
 - 0s - loss: 1.4653 - acc: 0.2143
Epoch 4/1000
 - 0s - loss: 1.4576 - acc: 0.2143
Epoch 5/1000
 - 0s - loss: 1.4502 - acc: 0.2143
Epoch 6/1000
 - 0s - loss: 1.4431 - acc: 0.2143
Epoch 7/1000
 - 0s - loss: 1.4333 - acc: 0.2143
Epoch 8/1000
 - 0s - loss: 1.4262 - acc: 0.2143
Epoch 9/1000
 - 0s - loss: 1.4205 - acc: 0.2143
Epoch 10/1000
 - 0s - loss: 1.4098 - acc: 0.2143
Epoch 11/1000
 - 0s - loss: 1.4056 - acc: 0.2143
Epoch 12/1000
 - 0s - loss: 1.3943 - acc: 0.2143
Epoch 13/1000
 - 0s - loss: 1.3858 - acc: 0.2143
Epoch 14/1000
 - 0s - loss: 1.3797 - acc: 0.2143
Epoch 15/1000
 - 0s - loss: 1.3697 - acc: 0.2143
Epoch 16/1000
 - 0s - loss: 1.3614 - acc: 0.2143
Epoch 17/1000
 - 0s - loss: 1.3537 - acc: 0.2143
Epoch 18/1000
 - 0s - loss: 1.3478 - acc: 0.2143
Epoch 19/1000
 - 0s - loss: 1.3389 - acc: 0.2143
Epoch 20/1000
 - 0s - loss: 1.3290 - acc: 0.2143
Epoch 21/1000
 - 0s - loss: 1

Epoch 168/1000
 - 0s - loss: 0.4781 - acc: 0.2857
Epoch 169/1000
 - 0s - loss: 0.4757 - acc: 0.2857
Epoch 170/1000
 - 0s - loss: 0.4722 - acc: 0.2857
Epoch 171/1000
 - 0s - loss: 0.4684 - acc: 0.2857
Epoch 172/1000
 - 0s - loss: 0.4654 - acc: 0.2857
Epoch 173/1000
 - 0s - loss: 0.4621 - acc: 0.2857
Epoch 174/1000
 - 0s - loss: 0.4583 - acc: 0.2857
Epoch 175/1000
 - 0s - loss: 0.4550 - acc: 0.2857
Epoch 176/1000
 - 0s - loss: 0.4532 - acc: 0.2857
Epoch 177/1000
 - 0s - loss: 0.4489 - acc: 0.3571
Epoch 178/1000
 - 0s - loss: 0.4456 - acc: 0.3571
Epoch 179/1000
 - 0s - loss: 0.4421 - acc: 0.3571
Epoch 180/1000
 - 0s - loss: 0.4390 - acc: 0.4286
Epoch 181/1000
 - 0s - loss: 0.4365 - acc: 0.4286
Epoch 182/1000
 - 0s - loss: 0.4331 - acc: 0.4286
Epoch 183/1000
 - 0s - loss: 0.4300 - acc: 0.4286
Epoch 184/1000
 - 0s - loss: 0.4272 - acc: 0.4286
Epoch 185/1000
 - 0s - loss: 0.4243 - acc: 0.4286
Epoch 186/1000
 - 0s - loss: 0.4213 - acc: 0.4286
Epoch 187/1000
 - 0s - loss: 0.4198 - acc: 0.4286


Epoch 332/1000
 - 0s - loss: 0.2018 - acc: 0.5714
Epoch 333/1000
 - 0s - loss: 0.2009 - acc: 0.5714
Epoch 334/1000
 - 0s - loss: 0.2002 - acc: 0.5714
Epoch 335/1000
 - 0s - loss: 0.1995 - acc: 0.5714
Epoch 336/1000
 - 0s - loss: 0.1988 - acc: 0.5714
Epoch 337/1000
 - 0s - loss: 0.1983 - acc: 0.5714
Epoch 338/1000
 - 0s - loss: 0.1974 - acc: 0.5714
Epoch 339/1000
 - 0s - loss: 0.1966 - acc: 0.5714
Epoch 340/1000
 - 0s - loss: 0.1959 - acc: 0.5714
Epoch 341/1000
 - 0s - loss: 0.1954 - acc: 0.5714
Epoch 342/1000
 - 0s - loss: 0.1947 - acc: 0.5714
Epoch 343/1000
 - 0s - loss: 0.1938 - acc: 0.5714
Epoch 344/1000
 - 0s - loss: 0.1933 - acc: 0.5714
Epoch 345/1000
 - 0s - loss: 0.1925 - acc: 0.5714
Epoch 346/1000
 - 0s - loss: 0.1921 - acc: 0.5714
Epoch 347/1000
 - 0s - loss: 0.1919 - acc: 0.5714
Epoch 348/1000
 - 0s - loss: 0.1910 - acc: 0.5714
Epoch 349/1000
 - 0s - loss: 0.1902 - acc: 0.5714
Epoch 350/1000
 - 0s - loss: 0.1895 - acc: 0.5714
Epoch 351/1000
 - 0s - loss: 0.1890 - acc: 0.5714


Epoch 496/1000
 - 0s - loss: 0.1200 - acc: 0.8571
Epoch 497/1000
 - 0s - loss: 0.1201 - acc: 0.8571
Epoch 498/1000
 - 0s - loss: 0.1199 - acc: 0.8571
Epoch 499/1000
 - 0s - loss: 0.1191 - acc: 0.8571
Epoch 500/1000
 - 0s - loss: 0.1187 - acc: 0.8571
Epoch 501/1000
 - 0s - loss: 0.1183 - acc: 0.8571
Epoch 502/1000
 - 0s - loss: 0.1182 - acc: 0.8571
Epoch 503/1000
 - 0s - loss: 0.1177 - acc: 0.8571
Epoch 504/1000
 - 0s - loss: 0.1173 - acc: 0.8571
Epoch 505/1000
 - 0s - loss: 0.1169 - acc: 0.8571
Epoch 506/1000
 - 0s - loss: 0.1165 - acc: 0.8571
Epoch 507/1000
 - 0s - loss: 0.1162 - acc: 0.8571
Epoch 508/1000
 - 0s - loss: 0.1164 - acc: 0.8571
Epoch 509/1000
 - 0s - loss: 0.1155 - acc: 0.8571
Epoch 510/1000
 - 0s - loss: 0.1154 - acc: 0.8571
Epoch 511/1000
 - 0s - loss: 0.1149 - acc: 0.8571
Epoch 512/1000
 - 0s - loss: 0.1145 - acc: 0.8571
Epoch 513/1000
 - 0s - loss: 0.1143 - acc: 0.8571
Epoch 514/1000
 - 0s - loss: 0.1138 - acc: 0.8571
Epoch 515/1000
 - 0s - loss: 0.1137 - acc: 0.8571


Epoch 660/1000
 - 0s - loss: 0.0734 - acc: 1.0000
Epoch 661/1000
 - 0s - loss: 0.0733 - acc: 1.0000
Epoch 662/1000
 - 0s - loss: 0.0733 - acc: 1.0000
Epoch 663/1000
 - 0s - loss: 0.0727 - acc: 1.0000
Epoch 664/1000
 - 0s - loss: 0.0727 - acc: 1.0000
Epoch 665/1000
 - 0s - loss: 0.0724 - acc: 1.0000
Epoch 666/1000
 - 0s - loss: 0.0722 - acc: 1.0000
Epoch 667/1000
 - 0s - loss: 0.0720 - acc: 1.0000
Epoch 668/1000
 - 0s - loss: 0.0717 - acc: 1.0000
Epoch 669/1000
 - 0s - loss: 0.0715 - acc: 1.0000
Epoch 670/1000
 - 0s - loss: 0.0713 - acc: 1.0000
Epoch 671/1000
 - 0s - loss: 0.0712 - acc: 1.0000
Epoch 672/1000
 - 0s - loss: 0.0710 - acc: 1.0000
Epoch 673/1000
 - 0s - loss: 0.0708 - acc: 1.0000
Epoch 674/1000
 - 0s - loss: 0.0705 - acc: 1.0000
Epoch 675/1000
 - 0s - loss: 0.0703 - acc: 1.0000
Epoch 676/1000
 - 0s - loss: 0.0701 - acc: 1.0000
Epoch 677/1000
 - 0s - loss: 0.0697 - acc: 1.0000
Epoch 678/1000
 - 0s - loss: 0.0696 - acc: 1.0000
Epoch 679/1000
 - 0s - loss: 0.0693 - acc: 1.0000


Epoch 824/1000
 - 0s - loss: 0.0446 - acc: 1.0000
Epoch 825/1000
 - 0s - loss: 0.0445 - acc: 1.0000
Epoch 826/1000
 - 0s - loss: 0.0444 - acc: 1.0000
Epoch 827/1000
 - 0s - loss: 0.0443 - acc: 1.0000
Epoch 828/1000
 - 0s - loss: 0.0442 - acc: 1.0000
Epoch 829/1000
 - 0s - loss: 0.0441 - acc: 1.0000
Epoch 830/1000
 - 0s - loss: 0.0441 - acc: 1.0000
Epoch 831/1000
 - 0s - loss: 0.0437 - acc: 1.0000
Epoch 832/1000
 - 0s - loss: 0.0436 - acc: 1.0000
Epoch 833/1000
 - 0s - loss: 0.0435 - acc: 1.0000
Epoch 834/1000
 - 0s - loss: 0.0433 - acc: 1.0000
Epoch 835/1000
 - 0s - loss: 0.0433 - acc: 1.0000
Epoch 836/1000
 - 0s - loss: 0.0432 - acc: 1.0000
Epoch 837/1000
 - 0s - loss: 0.0431 - acc: 1.0000
Epoch 838/1000
 - 0s - loss: 0.0428 - acc: 1.0000
Epoch 839/1000
 - 0s - loss: 0.0427 - acc: 1.0000
Epoch 840/1000
 - 0s - loss: 0.0426 - acc: 1.0000
Epoch 841/1000
 - 0s - loss: 0.0424 - acc: 1.0000
Epoch 842/1000
 - 0s - loss: 0.0423 - acc: 1.0000
Epoch 843/1000
 - 0s - loss: 0.0422 - acc: 1.0000


Epoch 988/1000
 - 0s - loss: 0.0270 - acc: 1.0000
Epoch 989/1000
 - 0s - loss: 0.0269 - acc: 1.0000
Epoch 990/1000
 - 0s - loss: 0.0268 - acc: 1.0000
Epoch 991/1000
 - 0s - loss: 0.0267 - acc: 1.0000
Epoch 992/1000
 - 0s - loss: 0.0267 - acc: 1.0000
Epoch 993/1000
 - 0s - loss: 0.0266 - acc: 1.0000
Epoch 994/1000
 - 0s - loss: 0.0265 - acc: 1.0000
Epoch 995/1000
 - 0s - loss: 0.0264 - acc: 1.0000
Epoch 996/1000
 - 0s - loss: 0.0263 - acc: 1.0000
Epoch 997/1000
 - 0s - loss: 0.0262 - acc: 1.0000
Epoch 998/1000
 - 0s - loss: 0.0261 - acc: 1.0000
Epoch 999/1000
 - 0s - loss: 0.0261 - acc: 1.0000
Epoch 1000/1000
 - 0s - loss: 0.0260 - acc: 1.0000


In [32]:
y = base_model.predict(data[1:15])
for i in range(len(y)):
    print(data[i+1], label[i+1], y[i])

[1. 1. 0. 0. 0.] -1.0 [-0.7302062]
[1. 0. 1. 0. 0.] 1.0 [0.9110876]
[1. 0. 0. 1. 0.] 1.0 [0.9110301]
[1. 0. 0. 0. 1.] -1.0 [-0.9953436]
[1. 1. 1. 0. 0.] 1.0 [0.78812593]
[1. 1. 0. 1. 0.] 1.0 [0.78799784]
[1. 1. 0. 0. 1.] -1.0 [-0.99816906]
[1. 0. 1. 1. 0.] 1.0 [0.9982811]
[1. 0. 1. 0. 1.] -1.0 [-0.7757178]
[1. 0. 0. 1. 1.] -1.0 [-0.77585226]
[1. 1. 1. 1. 0.] 1.0 [0.9956281]
[1. 1. 1. 0. 1.] -1.0 [-0.9054998]
[1. 1. 0. 1. 1.] -1.0 [-0.9055607]
[1. 0. 1. 1. 1.] 1.0 [0.7446197]


In [33]:
combined_model.fit([data[1:15], ht_input[1:15]], 
                   np_label[1:15], 
                   epochs=500,
                   batch_size=1,
                   verbose=2)

Epoch 1/500
 - 0s - loss: 0.3313 - acc: 0.3571
Epoch 2/500
 - 0s - loss: 0.3264 - acc: 0.3571
Epoch 3/500
 - 0s - loss: 0.3211 - acc: 0.3571
Epoch 4/500
 - 0s - loss: 0.3161 - acc: 0.3571
Epoch 5/500
 - 0s - loss: 0.3112 - acc: 0.3571
Epoch 6/500
 - 0s - loss: 0.3064 - acc: 0.3571
Epoch 7/500
 - 0s - loss: 0.3018 - acc: 0.3571
Epoch 8/500
 - 0s - loss: 0.2967 - acc: 0.3571
Epoch 9/500
 - 0s - loss: 0.2920 - acc: 0.3571
Epoch 10/500
 - 0s - loss: 0.2876 - acc: 0.3571
Epoch 11/500
 - 0s - loss: 0.2828 - acc: 0.3571
Epoch 12/500
 - 0s - loss: 0.2785 - acc: 0.3571
Epoch 13/500
 - 0s - loss: 0.2742 - acc: 0.3571
Epoch 14/500
 - 0s - loss: 0.2699 - acc: 0.3571
Epoch 15/500
 - 0s - loss: 0.2654 - acc: 0.3571
Epoch 16/500
 - 0s - loss: 0.2616 - acc: 0.3571
Epoch 17/500
 - 0s - loss: 0.2573 - acc: 0.3571
Epoch 18/500
 - 0s - loss: 0.2532 - acc: 0.3571
Epoch 19/500
 - 0s - loss: 0.2498 - acc: 0.3571
Epoch 20/500
 - 0s - loss: 0.2456 - acc: 0.3571
Epoch 21/500
 - 0s - loss: 0.2423 - acc: 0.3571
E

Epoch 171/500
 - 0s - loss: 0.0969 - acc: 0.7857
Epoch 172/500
 - 0s - loss: 0.0956 - acc: 0.7857
Epoch 173/500
 - 0s - loss: 0.0948 - acc: 0.7857
Epoch 174/500
 - 0s - loss: 0.0938 - acc: 0.7857
Epoch 175/500
 - 0s - loss: 0.0927 - acc: 0.7857
Epoch 176/500
 - 0s - loss: 0.0919 - acc: 0.7857
Epoch 177/500
 - 0s - loss: 0.0909 - acc: 0.7857
Epoch 178/500
 - 0s - loss: 0.0901 - acc: 0.7857
Epoch 179/500
 - 0s - loss: 0.0891 - acc: 0.7857
Epoch 180/500
 - 0s - loss: 0.0883 - acc: 0.7857
Epoch 181/500
 - 0s - loss: 0.0875 - acc: 0.7857
Epoch 182/500
 - 0s - loss: 0.0867 - acc: 0.7857
Epoch 183/500
 - 0s - loss: 0.0860 - acc: 0.7857
Epoch 184/500
 - 0s - loss: 0.0853 - acc: 0.7857
Epoch 185/500
 - 0s - loss: 0.0845 - acc: 0.7857
Epoch 186/500
 - 0s - loss: 0.0837 - acc: 0.7857
Epoch 187/500
 - 0s - loss: 0.0831 - acc: 0.7857
Epoch 188/500
 - 0s - loss: 0.0824 - acc: 0.7857
Epoch 189/500
 - 0s - loss: 0.0817 - acc: 1.0000
Epoch 190/500
 - 0s - loss: 0.0811 - acc: 1.0000
Epoch 191/500
 - 0s 

Epoch 339/500
 - 0s - loss: 0.0147 - acc: 1.0000
Epoch 340/500
 - 0s - loss: 0.0145 - acc: 1.0000
Epoch 341/500
 - 0s - loss: 0.0142 - acc: 1.0000
Epoch 342/500
 - 0s - loss: 0.0139 - acc: 1.0000
Epoch 343/500
 - 0s - loss: 0.0138 - acc: 1.0000
Epoch 344/500
 - 0s - loss: 0.0134 - acc: 1.0000
Epoch 345/500
 - 0s - loss: 0.0132 - acc: 1.0000
Epoch 346/500
 - 0s - loss: 0.0130 - acc: 1.0000
Epoch 347/500
 - 0s - loss: 0.0127 - acc: 1.0000
Epoch 348/500
 - 0s - loss: 0.0125 - acc: 1.0000
Epoch 349/500
 - 0s - loss: 0.0123 - acc: 1.0000
Epoch 350/500
 - 0s - loss: 0.0121 - acc: 1.0000
Epoch 351/500
 - 0s - loss: 0.0119 - acc: 1.0000
Epoch 352/500
 - 0s - loss: 0.0117 - acc: 1.0000
Epoch 353/500
 - 0s - loss: 0.0114 - acc: 1.0000
Epoch 354/500
 - 0s - loss: 0.0113 - acc: 1.0000
Epoch 355/500
 - 0s - loss: 0.0111 - acc: 1.0000
Epoch 356/500
 - 0s - loss: 0.0109 - acc: 1.0000
Epoch 357/500
 - 0s - loss: 0.0107 - acc: 1.0000
Epoch 358/500
 - 0s - loss: 0.0105 - acc: 1.0000
Epoch 359/500
 - 0s 

In [34]:
net_weight = combined_model.get_weights()
net_weight

[array([[-0.10182971],
        [-0.3855561 ],
        [ 2.3171604 ],
        [ 2.0895085 ],
        [-3.093571  ]], dtype=float32),
 array([-0.63574654], dtype=float32),
 array([[2.594885]], dtype=float32),
 array([[2.4404867]], dtype=float32),
 array([[2.2996926]], dtype=float32),
 array([[1.8188928]], dtype=float32),
 array([[-1.3551998],
        [ 1.4264638],
        [ 2.0924454],
        [-1.5386447]], dtype=float32)]

In [35]:
print('word_sign \t Tanh_w \t ReLu_w \t sigmoid_w \t word_importance')

for i in range(4):
    print(np.sign(word_weight[i+1]), '\t', net_weight[0].flatten()[i+1], 
          '\t', net_weight[i+2].flatten(), '\t',net_weight[6].flatten()[i], '\t',
         net_weight[i+2].flatten()*net_weight[6].flatten()[i])

word_sign 	 Tanh_w 	 ReLu_w 	 sigmoid_w 	 word_importance
-1 	 -0.3855561 	 [2.594885] 	 -1.3551998 	 [-3.5165877]
1 	 2.3171604 	 [2.4404867] 	 1.4264638 	 [3.481266]
1 	 2.0895085 	 [2.2996926] 	 2.0924454 	 [4.811981]
-1 	 -3.093571 	 [1.8188928] 	 -1.5386447 	 [-2.7986298]


In [36]:
def report():
    for i in range(1,15):
        bm = base_model.predict(np.reshape(data[i], (1,5)))
        
        cm = combined_model.predict([np.reshape(data[i], (1,5)), 
                                np.reshape(ht_input[i], (1,4))])
        
#         document_output = 'multiply'
#         document_predict = Model(inputs=combined_model.input,
#                                      outputs=combined_model.get_layer(document_output).output)
#         doc_output = document_predict.predict([np.reshape(data[i], (1,5)), 
#                                       ht_1_input[i], 
#                                       ht_2_input[i], 
#                                       ht_3_input[i], 
#                                       ht_4_input[i]])
        
        layer_name = 'concatenate'
        concat_after_relu = Model(inputs=combined_model.input,
                                     outputs=combined_model.get_layer(layer_name).output)
        concat_output = concat_after_relu.predict([np.reshape(data[i], (1,5)), 
                                     np.reshape(ht_input[i], (1,4))])
        
        print(data[i], '\t', np_label[i], '\t', bm.flatten(), '\t', cm.flatten(), '\t',concat_output.flatten())
    

In [37]:
ht_input.shape

(16, 4)

In [38]:
report()
# need to modify the ht_input

[1. 1. 0. 0. 0.] 	 0 	 [-0.8086557] 	 [0.05500743] 	 [2.0983686 0.        0.        0.       ]
[1. 0. 1. 0. 0.] 	 1 	 [0.9185369] 	 [0.9607466] 	 [0.       2.241677 0.       0.      ]
[1. 0. 0. 1. 0.] 	 1 	 [0.8745086] 	 [0.98534364] 	 [0.       0.       2.011101 0.      ]
[1. 0. 0. 0. 1.] 	 0 	 [-0.99906] 	 [0.05754076] 	 [0.       0.       0.       1.817183]
[1. 1. 1. 0. 0.] 	 1 	 [0.83182406] 	 [0.9476385] 	 [0.        2.0300555 0.        0.       ]
[1. 1. 0. 1. 0.] 	 1 	 [0.74710757] 	 [0.97327507] 	 [0.        0.        1.7181177 0.       ]
[1. 1. 0. 0. 1.] 	 0 	 [-0.99956506] 	 [0.00181026] 	 [2.5937564 0.        0.        1.8181018]
[1. 0. 1. 1. 0.] 	 1 	 [0.99870026] 	 [0.9997471] 	 [0.        2.4373147 2.2967036 0.       ]
[1. 0. 1. 0. 1.] 	 0 	 [-0.90764403] 	 [0.07309035] 	 [0.        0.        0.        1.6509073]
[1. 0. 0. 1. 1.] 	 0 	 [-0.94041646] 	 [0.06711479] 	 [0.        0.        0.        1.7105168]
[1. 1. 1. 1. 0.] 	 1 	 [0.9971921] 	 [0.999744] 	 [0.        2.433

In [39]:
word_weight

array([ 0, -1,  2,  3, -4])

In [ ]:
from keras.models import load_model

# base_model.save('overfit_base_model.h5')
# combined_model.save('overfit_combined_model.h5')

In [ ]:
# bm = base_model.predict(np.reshape(data[test_idx], (1,5)))
# print(bm)

# before jointly trained.
# 0.7995629

### IMDB network Model
#### concat and split layer test

In [40]:
# build the combined model
# Combined model
human_terms_len = 4

base_model = build_base_model(data.shape[1])

combined_input_layer = Input(shape=(data.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

# get the document prediction
label_layer = base_model(combined_input_layer)

# multiply and pass it into relu
# initialize relu layer

relu_layer = Dense(1, activation='relu', name='relu_layer', use_bias=False, kernel_initializer='ones')

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
    dense_layer.append(relu_layer(Multiply()([split[i], label_layer])))

# concat all the result   
concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 5)                 0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 4)            0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 5)            0                                            
_________________________________________________

In [41]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [42]:
base_model.fit(data[1:15], label[1:15], epochs=1000, batch_size=1, verbose=2)

Epoch 1/1000
 - 0s - loss: 0.5107 - acc: 0.2857
Epoch 2/1000
 - 0s - loss: 0.5044 - acc: 0.2857
Epoch 3/1000
 - 0s - loss: 0.4991 - acc: 0.2857
Epoch 4/1000
 - 0s - loss: 0.4932 - acc: 0.2857
Epoch 5/1000
 - 0s - loss: 0.4893 - acc: 0.3571
Epoch 6/1000
 - 0s - loss: 0.4838 - acc: 0.3571
Epoch 7/1000
 - 0s - loss: 0.4799 - acc: 0.3571
Epoch 8/1000
 - 0s - loss: 0.4756 - acc: 0.3571
Epoch 9/1000
 - 0s - loss: 0.4712 - acc: 0.3571
Epoch 10/1000
 - 0s - loss: 0.4674 - acc: 0.3571
Epoch 11/1000
 - 0s - loss: 0.4628 - acc: 0.3571
Epoch 12/1000
 - 0s - loss: 0.4587 - acc: 0.3571
Epoch 13/1000
 - 0s - loss: 0.4563 - acc: 0.3571
Epoch 14/1000
 - 0s - loss: 0.4517 - acc: 0.3571
Epoch 15/1000
 - 0s - loss: 0.4472 - acc: 0.3571
Epoch 16/1000
 - 0s - loss: 0.4442 - acc: 0.3571
Epoch 17/1000
 - 0s - loss: 0.4405 - acc: 0.3571
Epoch 18/1000
 - 0s - loss: 0.4377 - acc: 0.3571
Epoch 19/1000
 - 0s - loss: 0.4343 - acc: 0.3571
Epoch 20/1000
 - 0s - loss: 0.4304 - acc: 0.3571
Epoch 21/1000
 - 0s - loss: 0

Epoch 168/1000
 - 0s - loss: 0.1921 - acc: 0.7143
Epoch 169/1000
 - 0s - loss: 0.1914 - acc: 0.7143
Epoch 170/1000
 - 0s - loss: 0.1906 - acc: 0.7143
Epoch 171/1000
 - 0s - loss: 0.1895 - acc: 0.7143
Epoch 172/1000
 - 0s - loss: 0.1889 - acc: 0.7143
Epoch 173/1000
 - 0s - loss: 0.1880 - acc: 0.7143
Epoch 174/1000
 - 0s - loss: 0.1876 - acc: 0.7143
Epoch 175/1000
 - 0s - loss: 0.1866 - acc: 0.7143
Epoch 176/1000
 - 0s - loss: 0.1857 - acc: 0.7143
Epoch 177/1000
 - 0s - loss: 0.1852 - acc: 0.7143
Epoch 178/1000
 - 0s - loss: 0.1844 - acc: 0.7143
Epoch 179/1000
 - 0s - loss: 0.1839 - acc: 0.7143
Epoch 180/1000
 - 0s - loss: 0.1829 - acc: 0.7143
Epoch 181/1000
 - 0s - loss: 0.1820 - acc: 0.7143
Epoch 182/1000
 - 0s - loss: 0.1812 - acc: 0.7143
Epoch 183/1000
 - 0s - loss: 0.1806 - acc: 0.7143
Epoch 184/1000
 - 0s - loss: 0.1797 - acc: 0.7143
Epoch 185/1000
 - 0s - loss: 0.1796 - acc: 0.7143
Epoch 186/1000
 - 0s - loss: 0.1791 - acc: 0.7143
Epoch 187/1000
 - 0s - loss: 0.1776 - acc: 0.7143


Epoch 332/1000
 - 0s - loss: 0.1046 - acc: 1.0000
Epoch 333/1000
 - 0s - loss: 0.1040 - acc: 1.0000
Epoch 334/1000
 - 0s - loss: 0.1037 - acc: 1.0000
Epoch 335/1000
 - 0s - loss: 0.1034 - acc: 1.0000
Epoch 336/1000
 - 0s - loss: 0.1029 - acc: 1.0000
Epoch 337/1000
 - 0s - loss: 0.1026 - acc: 1.0000
Epoch 338/1000
 - 0s - loss: 0.1023 - acc: 1.0000
Epoch 339/1000
 - 0s - loss: 0.1020 - acc: 1.0000
Epoch 340/1000
 - 0s - loss: 0.1018 - acc: 1.0000
Epoch 341/1000
 - 0s - loss: 0.1014 - acc: 1.0000
Epoch 342/1000
 - 0s - loss: 0.1012 - acc: 1.0000
Epoch 343/1000
 - 0s - loss: 0.1006 - acc: 1.0000
Epoch 344/1000
 - 0s - loss: 0.1003 - acc: 1.0000
Epoch 345/1000
 - 0s - loss: 0.0999 - acc: 1.0000
Epoch 346/1000
 - 0s - loss: 0.0996 - acc: 1.0000
Epoch 347/1000
 - 0s - loss: 0.0993 - acc: 1.0000
Epoch 348/1000
 - 0s - loss: 0.0989 - acc: 1.0000
Epoch 349/1000
 - 0s - loss: 0.0986 - acc: 1.0000
Epoch 350/1000
 - 0s - loss: 0.0983 - acc: 1.0000
Epoch 351/1000
 - 0s - loss: 0.0980 - acc: 1.0000


Epoch 496/1000
 - 0s - loss: 0.0608 - acc: 1.0000
Epoch 497/1000
 - 0s - loss: 0.0604 - acc: 1.0000
Epoch 498/1000
 - 0s - loss: 0.0603 - acc: 1.0000
Epoch 499/1000
 - 0s - loss: 0.0600 - acc: 1.0000
Epoch 500/1000
 - 0s - loss: 0.0600 - acc: 1.0000
Epoch 501/1000
 - 0s - loss: 0.0597 - acc: 1.0000
Epoch 502/1000
 - 0s - loss: 0.0595 - acc: 1.0000
Epoch 503/1000
 - 0s - loss: 0.0592 - acc: 1.0000
Epoch 504/1000
 - 0s - loss: 0.0590 - acc: 1.0000
Epoch 505/1000
 - 0s - loss: 0.0589 - acc: 1.0000
Epoch 506/1000
 - 0s - loss: 0.0587 - acc: 1.0000
Epoch 507/1000
 - 0s - loss: 0.0585 - acc: 1.0000
Epoch 508/1000
 - 0s - loss: 0.0583 - acc: 1.0000
Epoch 509/1000
 - 0s - loss: 0.0583 - acc: 1.0000
Epoch 510/1000
 - 0s - loss: 0.0580 - acc: 1.0000
Epoch 511/1000
 - 0s - loss: 0.0578 - acc: 1.0000
Epoch 512/1000
 - 0s - loss: 0.0576 - acc: 1.0000
Epoch 513/1000
 - 0s - loss: 0.0575 - acc: 1.0000
Epoch 514/1000
 - 0s - loss: 0.0573 - acc: 1.0000
Epoch 515/1000
 - 0s - loss: 0.0571 - acc: 1.0000


Epoch 660/1000
 - 0s - loss: 0.0359 - acc: 1.0000
Epoch 661/1000
 - 0s - loss: 0.0356 - acc: 1.0000
Epoch 662/1000
 - 0s - loss: 0.0356 - acc: 1.0000
Epoch 663/1000
 - 0s - loss: 0.0354 - acc: 1.0000
Epoch 664/1000
 - 0s - loss: 0.0353 - acc: 1.0000
Epoch 665/1000
 - 0s - loss: 0.0353 - acc: 1.0000
Epoch 666/1000
 - 0s - loss: 0.0351 - acc: 1.0000
Epoch 667/1000
 - 0s - loss: 0.0350 - acc: 1.0000
Epoch 668/1000
 - 0s - loss: 0.0349 - acc: 1.0000
Epoch 669/1000
 - 0s - loss: 0.0348 - acc: 1.0000
Epoch 670/1000
 - 0s - loss: 0.0347 - acc: 1.0000
Epoch 671/1000
 - 0s - loss: 0.0345 - acc: 1.0000
Epoch 672/1000
 - 0s - loss: 0.0344 - acc: 1.0000
Epoch 673/1000
 - 0s - loss: 0.0343 - acc: 1.0000
Epoch 674/1000
 - 0s - loss: 0.0342 - acc: 1.0000
Epoch 675/1000
 - 0s - loss: 0.0341 - acc: 1.0000
Epoch 676/1000
 - 0s - loss: 0.0341 - acc: 1.0000
Epoch 677/1000
 - 0s - loss: 0.0338 - acc: 1.0000
Epoch 678/1000
 - 0s - loss: 0.0338 - acc: 1.0000
Epoch 679/1000
 - 0s - loss: 0.0338 - acc: 1.0000


Epoch 824/1000
 - 0s - loss: 0.0212 - acc: 1.0000
Epoch 825/1000
 - 0s - loss: 0.0211 - acc: 1.0000
Epoch 826/1000
 - 0s - loss: 0.0210 - acc: 1.0000
Epoch 827/1000
 - 0s - loss: 0.0209 - acc: 1.0000
Epoch 828/1000
 - 0s - loss: 0.0209 - acc: 1.0000
Epoch 829/1000
 - 0s - loss: 0.0208 - acc: 1.0000
Epoch 830/1000
 - 0s - loss: 0.0208 - acc: 1.0000
Epoch 831/1000
 - 0s - loss: 0.0207 - acc: 1.0000
Epoch 832/1000
 - 0s - loss: 0.0207 - acc: 1.0000
Epoch 833/1000
 - 0s - loss: 0.0206 - acc: 1.0000
Epoch 834/1000
 - 0s - loss: 0.0205 - acc: 1.0000
Epoch 835/1000
 - 0s - loss: 0.0204 - acc: 1.0000
Epoch 836/1000
 - 0s - loss: 0.0203 - acc: 1.0000
Epoch 837/1000
 - 0s - loss: 0.0203 - acc: 1.0000
Epoch 838/1000
 - 0s - loss: 0.0202 - acc: 1.0000
Epoch 839/1000
 - 0s - loss: 0.0202 - acc: 1.0000
Epoch 840/1000
 - 0s - loss: 0.0202 - acc: 1.0000
Epoch 841/1000
 - 0s - loss: 0.0200 - acc: 1.0000
Epoch 842/1000
 - 0s - loss: 0.0200 - acc: 1.0000
Epoch 843/1000
 - 0s - loss: 0.0199 - acc: 1.0000


Epoch 988/1000
 - 0s - loss: 0.0125 - acc: 1.0000
Epoch 989/1000
 - 0s - loss: 0.0125 - acc: 1.0000
Epoch 990/1000
 - 0s - loss: 0.0124 - acc: 1.0000
Epoch 991/1000
 - 0s - loss: 0.0124 - acc: 1.0000
Epoch 992/1000
 - 0s - loss: 0.0123 - acc: 1.0000
Epoch 993/1000
 - 0s - loss: 0.0123 - acc: 1.0000
Epoch 994/1000
 - 0s - loss: 0.0123 - acc: 1.0000
Epoch 995/1000
 - 0s - loss: 0.0123 - acc: 1.0000
Epoch 996/1000
 - 0s - loss: 0.0122 - acc: 1.0000
Epoch 997/1000
 - 0s - loss: 0.0122 - acc: 1.0000
Epoch 998/1000
 - 0s - loss: 0.0121 - acc: 1.0000
Epoch 999/1000
 - 0s - loss: 0.0121 - acc: 1.0000
Epoch 1000/1000
 - 0s - loss: 0.0120 - acc: 1.0000


In [ ]:
combined_model.fit([data[1:15], ht_input[1:15]], 
                   np_label[1:15], 
                   epochs=500,
                   batch_size=1,
                   verbose=2)

In [ ]:
report()

In [43]:
def np_present_by_doc(X, human_terms, index, weight):
    np_vector = np.zeros([X.shape[0],])
    
    for i, x in enumerate(X):
        if x[index] == 1:
            np_vector[i] = np.sign(weight)
    
    return np_vector

In [44]:
def load_unigrams(path, X, y):
    word_list = []
    connotation = {}
    
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            word_list.append(line.strip())
            
    for word in word_list:
        pos_count = 0
        neg_count = 0
        for i, doc in enumerate(X):
            if word in doc.lower():
                if (y[i] == 1):
                    pos_count += 1
                else:
                    neg_count += 1
                    
        if pos_count > neg_count:
            connotation[word] = 1
        else:
            connotation[word] = 0
    
    return word_list, connotation

def generate_appearance(X_train_corpus, X_test_corpus, word_list, connotation):
    y_train_agreement = []
    for i in range(len(X_train_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_train_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_train_agreement.append(doc_agreement)
        
    y_test_agreement = []
    for i in range(len(X_test_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_test_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_test_agreement.append(doc_agreement)
        
    return np.array(y_train_agreement), np.array(y_test_agreement)

# 'imdb-unigrams.txt'

### Now try on IMDB data
#### Like the real IMDB

In [47]:
# load data
# make sure that the first shape is the IMDB training data. 

def open_pickle(path):
    import pickle
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train_original = open_pickle('../../data/imdb/imdb_original_preprocessed_xtrain.pickle')
X_test_original = open_pickle('../../data/imdb/imdb_original_preprocessed_xtest.pickle')
y_train_original = open_pickle('../../data/imdb/imdb_original_preprocessed_ytrain.pickle')
y_test_original = open_pickle('../../data/imdb/imdb_original_preprocessed_ytest.pickle')

In [48]:
# Count vectorizer 

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df = 100)
X_train = cv.fit_transform(X_train_original)
X_test = cv.transform(X_test_original)

In [49]:
word_list, connotation = load_unigrams('./imdb-unigrams.txt', X_train_original, y_train_original)

In [50]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_original, X_test_original, 
                                                          word_list, connotation)

In [51]:
y_train_agreement[:,1]

array([0, 0, 0, ..., 0, 0, 0])

In [52]:
len(word_list)

83

In [53]:
# build the combined model
# Combined model
human_terms_len = len(word_list)

base_model = build_base_model(X_train.shape[1])

combined_input_layer = Input(shape=(X_train.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

# get the document prediction
label_layer = base_model(combined_input_layer)

# multiply and pass it into relu
# initialize relu layer

relu_layer = Dense(1, activation='relu', name='relu_layer', use_bias=False, kernel_initializer='ones')

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
    dense_layer.append(relu_layer(Multiply()([split[i], label_layer])))

# concat all the result   
concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 3641)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3642      
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 83)           0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 3641)         0                                            
_________________________________________

In [56]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [57]:
y_train_tanh = y_train_original
y_train_tanh[y_train_tanh == 0] = -1

In [58]:
base_model.fit(X_train, y_train_tanh, batch_size=1, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 27s 1ms/step - loss: 0.4482 - acc: 0.7250
Epoch 2/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.3393 - acc: 0.8346
Epoch 3/10
25000/25000 [==============================] - 28s 1ms/step - loss: 0.3063 - acc: 0.8583
Epoch 4/10
25000/25000 [==============================] - 34s 1ms/step - loss: 0.2879 - acc: 0.8701
Epoch 5/10
25000/25000 [==============================] - 55s 2ms/step - loss: 0.2683 - acc: 0.8843
Epoch 6/10
25000/25000 [==============================] - 65s 3ms/step - loss: 0.2582 - acc: 0.8898
Epoch 7/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.2473 - acc: 0.8960
Epoch 8/10
25000/25000 [==============================] - 30s 1ms/step - loss: 0.2373 - acc: 0.9006
Epoch 9/10
25000/25000 [==============================] - 25s 1ms/step - loss: 0.2295 - acc: 0.9069
Epoch 10/10
25000/25000 [==============================] - 27s 1ms/step - loss: 0.2253 - acc: 0.9102

In [59]:
combined_model.fit([X_train,
                   y_train_agreement], y_train_original,
                  batch_size=1, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 70s 3ms/step - loss: 0.6972 - acc: 0.3758
Epoch 2/10
25000/25000 [==============================] - 77s 3ms/step - loss: 0.6653 - acc: 0.3855
Epoch 3/10
 5307/25000 [=====>........................] - ETA: 57s - loss: 0.6599 - acc: 0.3895

KeyboardInterrupt: 